In [15]:
import tensorflow as tf
# file list
filenames = ['./cameraman.png','./emoji_1.png']
# file queue
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.WholeFileReader()
_, value = reader.read(filename_queue)
# image queue
images = tf.image.decode_png(value)
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Finish off the filename queue coordinator.
    sess.run(tf.global_variables_initializer())
    for i in range(5):
        print(images.eval().shape)
    
    coord.request_stop()
    coord.join(threads)

(256, 256, 1)
(429, 705, 3)
(429, 705, 3)
(256, 256, 1)
(429, 705, 3)


In [13]:
import tensorflow as tf
# file list
filenames = ['./titanic_dataset.csv']
# file queue
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.TextLineReader(skip_header_lines=True)
_,value = reader.read(filename_queue)
record_defaults = [[1.],[1.],["None"],["None"],[1.],[1.],[1.],["None"],[1.0]]

label,pclass,name,sex,age,sibsp,parch,ticket,fare \
    = tf.decode_csv(value,record_defaults=record_defaults)
# Each variable is of Tensor type
# preprocess,将Tensor全部处理为实数
# 可以这样写,也可以如下写
#sex = tf.cast(tf.equal(sex,"female"),tf.float32) 
sex = tf.cond(tf.equal(sex,"female"),lambda: tf.constant(1.0),lambda: tf.constant(0.0))

feature = tf.stack([pclass,sex,age,sibsp,parch,fare])
label_batch,feature_batch = tf.train.shuffle_batch([label,feature],batch_size=10,min_after_dequeue=30,capacity=100)
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(1):
        print(feature_batch.eval())
    coord.request_stop()
    coord.join(threads)

[[   1.            1.           35.            0.            0.
   135.63330078]
 [   1.            0.           37.            1.            1.
    52.55419922]
 [   1.            0.           48.            0.            0.
    26.54999924]
 [   1.            1.           50.            0.            1.
   247.52079773]
 [   1.            0.            0.91670001    1.            2.
   151.55000305]
 [   1.            1.           30.            0.            0.
   164.86669922]
 [   1.            1.           63.            1.            0.
    77.95829773]
 [   1.            0.            0.            0.            0.
    25.92499924]
 [   1.            1.           24.            0.            0.
    69.30000305]
 [   1.            1.            2.            1.            2.
   151.55000305]]


In [20]:
import tensorflow as tf
import numpy as np
data = 10 * np.random.randn(100, 4) + 1
target = np.random.randint(0, 2, size=100) 
NUM_THREADS=4
# Queue
# Define a Queue
queue = tf.FIFOQueue(capacity=50 , dtypes = [tf.float32,tf.int32])
# Define enqueue operator
enqueue_op = queue.enqueue_many([data,target])
# Define dequeue operator
data_sample, label_sample = queue.dequeue()
# Define QueueRunner
qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in range(10):
        if coord.should_stop():
            break
        data_batch, label_batch = sess.run([data_sample, label_sample])
        print(data_batch)
        print(label_batch)
    coord.request_stop()
    coord.join(enqueue_threads)

[ -8.70554733   6.971663    13.07059669   1.6656239 ]
0
[-10.97807503  20.19344711  -7.4857378   25.31876755]
1
[  6.25867414  16.67843437   4.4450202    0.93004423]
1
[ -6.80075693   6.38164759  -8.53183556  18.01395988]
1
[-17.557024     9.95761585 -15.66968727 -12.46906662]
0
[-3.93508029 -2.41764307 -3.8494494   4.14832401]
1
[ -6.88991308  -3.17895055   5.85810137  23.6975956 ]
0
[ -1.70397878   4.10538292 -10.29130459  -4.23224545]
0
[  7.15477371 -20.28168297   9.09631538  -4.42101288]
0
[ -4.98077774  -5.920578    14.96670532   6.50292635]
1
